In [0]:
%run /Users/<user_email_address>/nyc_taxi/function_definitions

In [0]:
dlt_schema_name = "nyctaxi_dlt"
dlt_ingested_tbl_name = "nyctaxidata_latest_batch_of_ingested_trips_stg"
full_ingested_tbl_name = f"{dlt_schema_name}.{dlt_ingested_tbl_name}"

In [0]:
# Read from the staging table into a dataframe
trips_df_enhanced = (
    spark.read
    .format("delta")
    .table(full_ingested_tbl_name)
)

# Aggregate trips by zip codes
trips_df_agg = aggregate_by_zip_codes(
    trips_df_enhanced,
    100
).withColumn("batch_aggregation_timestamp", sf.current_timestamp())

# Write only the latest batch of aggregated data into a delta table
trips_df_agg.write.mode("overwrite").saveAsTable("nyctaxi_dlt.trips_agg_by_zip_codes_latest_batch")
# Truncate the staging table
trips_df_enhanced.limit(0).write.mode("overwrite").saveAsTable(full_ingested_tbl_name)